# Ramp of the simulation

In [1]:
#-----Importa paquetes escenciales

from __future__ import division
import hoomd
import hoomd.md

#-----Define las variables importantes

t_prev = 9.0
t_new = 9.5
pressure = 0.1

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_9.0_0.1_stabilization.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integra usando un NPT

temp = hoomd.variant.linear_interp(points = [(0,t_prev), (5e4, t_new)])
npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 2.5, tauP = 1.5, P = pressure);

#-----Guarda la información de la simulación en un archivo .log y en un .gsd

log_file = "T_" + str(t_new) + "_" + str(pressure) + "_ramp.log"
gsd_file = "T_" + str(t_new) + "_" + str(pressure) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_new) + "_" + str(pressure) + "_ramp.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
hoomd.run(5e4)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 2125508 / 2175000 | TPS 50.7807 | ETA 00:16:14
Time 00:00:20 | Step 2126109 / 2175000 | TPS 60.0478 | ETA 00:13:34
Time 00:00:30 | Step 2126741 / 2175000 | TPS 63.1571 | ETA 00:12:44
Time 00:00:40 | Step 2127335 / 2175000 | TPS 59.2608 | ETA 00:13:24
Time 00:00:50 | Step 2127873 / 2175000 | TPS 53.6737 | ETA 00:14:38
Time 00:01:00 | Step 2128429 / 2175000 | TPS 55.5457 | ETA 00:13:58
Time 00:01:10 | Step 2129010 / 2175000 | TPS 58.0827 | ETA 00:13:11
Time 00:01:20 | Step 2129611 / 2175000 | TPS 60.0574 | ETA 00:12:35
Time 00:01:30 | Step 2130213 / 2175000 | TPS 60.0582 | ETA 00:12:25
Time 00:01:40 | Step 2130764 / 2175000 | TPS 55.0961 | ETA 00:13:22
Time 00:01:50 | Step 2131334 / 2175000 | TPS 56.9825 | ETA 00:12:46
Time 00:0